# Analysis V4

We aim to define a response variable where knowing the value of it will be profitable to us. We 
then aim to fit a statistical model around this response variable such that we can predict it 
ahead of time, and make trades accordingly.

---

### Engineering a Response Variable

Our response variable is computed by return = (Price in n days) / (Price Today) for some n

In [23]:
from pandas import DataFrame, Series
from typing import List, Dict
from numpy import ndarray

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller
import itertools
import json
import math

response_lookforward: int = 10
commission_fee: float = 0.0005
CUTOFF: int = 60


In [24]:
def get_non_stationary_instruments() -> Dict[int, DataFrame]:
	raw_prices: DataFrame = pd.read_csv("../../prices.txt", sep=r"\s+", index_col=None, header=None)
	price_history: ndarray = raw_prices.values[:550][:].T
	data: Dict[int, DataFrame] = {}
	
	for instrument_no in range(0, 50):
		result = adfuller(price_history[instrument_no], autolag="AIC")
		
		if result[1] > 0.05:
			data[instrument_no] = pd.DataFrame(columns=["price"])
			data[instrument_no]["price"] = price_history[instrument_no]
	
	return data

def implement_response_variable(data: Dict[int, DataFrame]) -> Dict[int, DataFrame]:
	for instrument_no in data:
		data[instrument_no]["price_future"] = data[instrument_no]["price"].shift(
			-response_lookforward)
		
		data[instrument_no]["response"] = np.where(data[instrument_no]["price_future"].isna(), 
			0, np.where(data[instrument_no]["price_future"] > data[instrument_no]["price"], 1, 0)) 
	
	return data

# Backtesting the response variable
def implement_response_signals(data: Dict[int, DataFrame]) -> Dict[int, DataFrame]:
	for instrument_no in data:
		data[instrument_no]["signal"] = data[instrument_no]["response"]
		data[instrument_no]["signal"][data[instrument_no]["signal"] == 0] = -1
	
	return data

def get_strategy_results(data: Dict[int, DataFrame]) -> Dict[int, DataFrame]:
	for instrument_no in data:
		# Get Log Returns
		data[instrument_no]["log_return"] = np.log(data[instrument_no]
		["price"]).diff().shift(-1)

		# Get Strategy Return
		data[instrument_no]["strategy_return"] = (data[instrument_no]["signal"]
												  * data[instrument_no]["log_return"])

		# Get Position changes
		position_change: ndarray = data[instrument_no]["signal"].diff().abs()

		# Apply the commission fee
		data[instrument_no]["strategy_return"] -= position_change * commission_fee


	return data

def show_performance_metrics(data: Dict[int, DataFrame]) -> None:
	performance_metrics: Dict[str, List[int | float]] = {}
	performance_metrics["Instrument No."] = list(data.keys())
	performance_metrics["Profit Factor"] = []
	performance_metrics["Sharpe Ratio"] = []

	for instrument_no in data:
		# Get Returns
		returns: Series = data[instrument_no]["strategy_return"]

		# Compute performance metrics
		profit_factor = returns[returns > 0].sum() / returns[returns < 0].abs().sum()
		sharpe = (returns.mean() / returns.std()) * (252 ** 0.5)

		performance_metrics["Profit Factor"].append(profit_factor)
		performance_metrics["Sharpe Ratio"].append(sharpe)

	performance_metrics_df: DataFrame = pd.DataFrame(performance_metrics)
	print(performance_metrics_df.to_string(index=False))

def check_response_validity() -> None:		
	data: Dict[int, DataFrame] = get_non_stationary_instruments()
	data = implement_response_variable(data)
	data = implement_response_signals(data)
	data = get_strategy_results(data)
	show_performance_metrics(data)
	
check_response_validity()


 Instrument No.  Profit Factor  Sharpe Ratio
              0        1.65550       3.20301
              1        1.84633       3.90747
              2        1.87823       4.02344
              3        2.04167       4.48399
              4        1.76357       3.51197
              5        1.87993       4.00742
              6        2.22100       5.00050
              7        1.64218       3.10717
              8        1.81764       3.84325
              9        2.11175       4.61407
             10        1.79478       3.64333
             11        1.60355       2.97116
             12        1.89188       4.01491
             13        1.93538       4.20269
             14        2.10533       4.75445
             15        1.55782       2.74529
             16        1.94822       4.25179
             17        1.70666       3.42261
             19        1.61674       3.02271
             20        1.67801       3.32565
             21        1.94071       4.23561
          

/var/folders/xt/hd6p1tbs32g_7c8sy6ns0vc00000gn/T/ipykernel_41459/1445562490.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[instrument_no]["signal"][data[instrument_no]["signal"] == 0] = -1
/var/folders/xt/hd6p1tbs32g_7c8sy6ns0vc00000gn/T/ipykernel_41459/1445562490.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[instrument_no]["signal"][data[instrument_no]["signal"] == 0] = -1
/var/folders/xt/hd6p1tbs32g_7c8sy6ns0vc00000gn/T/ipykernel_41459/1445562490.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

### Feature Engineerring

In [25]:
def implement_rsi(data: Dict[int, DataFrame], params: Dict[int, Dict[str, float | int]]) -> (
		Dict)[int, DataFrame]:
	for instrument_no in data:
		delta_prices: Series = data[instrument_no]["price"].diff()

		gains: Series= delta_prices.clip(lower=0)
		losses: Series = -delta_prices.clip(upper=0)

		avg_gain: Series = gains.rolling(window=params[instrument_no]["rsi_lookback"]).mean()
		avg_loss: Series = losses.rolling(window=params[instrument_no]["rsi_lookback"]).mean()

		relative_strength: Series = avg_gain / avg_loss
		relative_strength_index: Series = 100 - (100 / (1 + relative_strength))
		relative_strength_index.iloc[:params[instrument_no]["rsi_lookback"]] = 0.0
		data[instrument_no]["rsi"] = relative_strength_index

	return data

def implement_volatility(data: Dict[int, DataFrame], params: Dict[int, Dict[str, float | int]]) -> (
		Dict)[int, DataFrame]:
	for instrument_no in data:
		prices_window: Series= data[instrument_no]["price"]
		returns: Series = prices_window.pct_change()
		volatility: Series = returns.rolling(window=params[instrument_no]["vol_lookback"]).std()
		data[instrument_no]["volatility"] = volatility.iloc[CUTOFF:]

	return data

# Positive: Slow EMA > Fast EMA - Bearish
# Negative: Slow EMA < Fast EMA - Bullish
def implement_ema_crossover(data: Dict[int, DataFrame], params: Dict[int, Dict[str, float | int]]
) -> Dict[int, DataFrame]:
	for instrument_no in data:
		prices_window: Series = data[instrument_no]["price"]
		slow_lookback: int = params[instrument_no]["slow_ema_lookback"]
		fast_lookback: int = params[instrument_no]["fast_ema_lookback"]

		slow_ema: Series = prices_window.ewm(span=slow_lookback, adjust=False).mean()
		fast_ema: Series = prices_window.ewm(span=fast_lookback, adjust=False).mean()

		crossover_gap: Series = slow_ema - fast_ema
		crossover_gap.iloc[:slow_lookback] = 0.0
		data[instrument_no]["ema_crossover"] = crossover_gap

	return data

test_params: Dict[str, int]= {
	"rsi_lookback": 14,
	"vol_lookback": 10,
	"fast_ema_lookback": 5,
	"slow_ema_lookback": 50
}

def setup_data(params: Dict[int, Dict[str, int]]) -> Dict[int, DataFrame]:
	data: Dict[int, DataFrame] = get_non_stationary_instruments()
	data = implement_response_variable(data)
	data = implement_rsi(data, params)
	data = implement_volatility(data, params)
	data = implement_ema_crossover(data, params)
	
	for instrument_no in data: data[instrument_no] = (data[instrument_no]
													  .iloc[CUTOFF:-response_lookforward]
													  .reset_index(drop=True))
	return data

### Checking the Linearity of the Logit with every predictor, as well as Collinearity and Perfect
 Separation

In [27]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

params_grid: Dict[str, List[int]] = {
	"rsi_lookback": [7, 14, 21],
	"vol_lookback": [10, 20, 30],
	"slow_ema_lookback": [5, 7, 10, 12],
	"fast_ema_lookback": [20, 30, 40, 50]
}

def perform_binned_logit_check() -> None: 
	keys = list(params_grid.keys())
	values = list(params_grid.values())
	valid_param_sets: Dict[int, List[Dict[str, List[int] | bool]]] = {
		instrument_no: [] for instrument_no in get_non_stationary_instruments()
	}

	total_combinations = math.prod(len(v) for v in values)
	print(f"Total combinations to try: {total_combinations}")
	
	for combination in itertools.product(*values):
		params = dict(zip(keys,combination))
		
		param_set: Dict[int, Dict[str, int]] = {
			instrument_no: params for instrument_no in get_non_stationary_instruments()
		}
		
		# Grab Data
		data: Dict[int, DataFrame] = setup_data(param_set)
		
		for instrument_no in data:
			# Augment each predictor and clip negative ones
			eps: float = 1e-8
			clipped_rsi = data[instrument_no]["rsi"].clip(lower=eps)
			data[instrument_no]["augmented_rsi"] = clipped_rsi * np.log(clipped_rsi)
			data[instrument_no]["augmented_vol"] = (data[instrument_no]["volatility"] * np.log
				(data[instrument_no]["volatility"]))
			clipped_ema: Series = data[instrument_no]["ema_crossover"].clip(lower=eps)
			data[instrument_no]["augmented_ema"] = (clipped_ema * np.log(clipped_ema))
			
			# Grab predictors and response
			predictors = (data[instrument_no].drop(columns=["response", "price", "price_future"])
						  .values)
			response = data[instrument_no]["response"].values
			
			# Scale each predictor so that they are equal in numerical scale
			scaler = StandardScaler()
			predictors_scaled = scaler.fit_transform(predictors)
			
			# If it detects collinearity, drop the combination
			cond = np.linalg.cond(predictors_scaled)
			if cond > 1e12: continue
			
			predictors_design = sm.add_constant(predictors_scaled)
			model = sm.Logit(response, predictors_design).fit(disp=False)
			
			# If the model didn't converge, just discard it
			converged = model.mle_retvals.get('converged', True)
			if not converged: continue
			
			# Check for p-values: for each augmented feature. If p value is high, drop the augmented
			# feature. If p value is low, keep the augmented feature
			valid_instrument_params = param_set[instrument_no]
			
			if model.pvalues[3] > 0.05:
				valid_instrument_params["augmented_rsi"] = False
			else:
				valid_instrument_params["augmented_rsi"] = True
			
			if model.pvalues[4] > 0.05:
				valid_instrument_params["augmented_vol"] = False
			else:
				valid_instrument_params["augmented_vol"] = True
				
			if model.pvalues[5] > 0.05:
				valid_instrument_params["augmented_ema"] = False
			else:
				valid_instrument_params["augmented_ema"] = True
				
			valid_param_sets[instrument_no].append(valid_instrument_params)
	
	# Save the valid parameters to a json file
	with open("valid_parameters.json", "w") as config_file:
		json.dump(valid_param_sets, config_file)
	
	config_file.close()
		
perform_binned_logit_check()

Total combinations to try: 144
